In [1]:
import pandas as pd
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt

# Load Dataset

In [2]:
df = pd.read_csv('./heart-failure.csv', sep=',', index_col=False)
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


# Function

In [3]:
def df_train(number_register):
    df_train = df.copy()
    df_train = df_train.head(number_register)
    return df_train

def df_test(number_register):
    df_test = df.copy()
    df_test = df_test.tail(number_register)
    return df_test

def min_value(column_name):
    min_value = df[column_name].min()
    print(f"El valor min {column_name} es {min_value}")
    return min_value

def max_value(column_name):
    max_value = df[column_name].max()
    print(f"El valor max {column_name} es {max_value}")
    return max_value

def data_gen_min_value(column_name):
    max_value = sintetico[column_name].min()
    print(f"El valor min {column_name} es {max_value}")
    return max_value

def data_gen_max_value(column_name):
    max_value = sintetico[column_name].max()
    print(f"El valor max {column_name} es {max_value}")
    return max_value

# Clean Data

In [4]:
# Nulos en las columnas
df_null = df.isnull().sum()
df_null

age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64

In [5]:
# int 
df[['age','platelets']] = df[['age','platelets']].astype(int)
#df

In [6]:
# rename target variable
df = df.rename(columns={'DEATH_EVENT':'target'})

In [7]:
# verifiying int in df
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,target
0,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
1,55,0,7861,0,38,0,263358,1.1,136,1,0,6,1
2,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
3,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
4,65,1,160,1,20,0,327000,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62,0,61,1,38,1,155000,1.1,143,1,1,270,0
295,55,0,1820,0,38,0,270000,1.2,139,0,0,271,0
296,45,0,2060,1,60,0,742000,0.8,138,0,0,278,0
297,45,0,2413,0,38,0,140000,1.4,140,1,1,280,0


# Analize data real

### Numeric columns

In [8]:
min_value_age = min_value('age')
max_value_age = max_value('age')

El valor min age es 40
El valor max age es 95


In [9]:
min_value_creatine = min_value('creatinine_phosphokinase')
max_value_creatine = max_value('creatinine_phosphokinase')

El valor min creatinine_phosphokinase es 23
El valor max creatinine_phosphokinase es 7861


In [10]:
min_value_ejection = min_value('ejection_fraction')
max_value_ejection = max_value('ejection_fraction')

El valor min ejection_fraction es 14
El valor max ejection_fraction es 80


In [11]:
min_value_platelets = min_value('platelets')
max_value_platelets = max_value('platelets')

El valor min platelets es 25100
El valor max platelets es 850000


In [12]:
min_value_creatinine = min_value('serum_creatinine')
max_value_creatinine = max_value('serum_creatinine')

El valor min serum_creatinine es 0.5
El valor max serum_creatinine es 9.4


In [13]:
min_value_sodium = min_value('serum_sodium')
max_value_sodium = max_value('serum_sodium')

El valor min serum_sodium es 113
El valor max serum_sodium es 148


In [14]:
min_value_time = min_value('time')
max_value_time = max_value('time')

El valor min time es 4
El valor max time es 285


### Binary columns

In [15]:
min_value_anemia = min_value('anaemia')
max_value_anemia = max_value('anaemia')

El valor min anaemia es 0
El valor max anaemia es 1


In [16]:
min_value_diabetes = min_value('diabetes')
max_value_diabetes = max_value('diabetes')

El valor min diabetes es 0
El valor max diabetes es 1


In [17]:
min_value_pressure = min_value('high_blood_pressure')
max_value_pressure = max_value('high_blood_pressure')

El valor min high_blood_pressure es 0
El valor max high_blood_pressure es 1


In [18]:
min_value_sex = min_value('sex')
max_value_sex = max_value('sex')

El valor min sex es 0
El valor max sex es 1


In [19]:
min_value_smoking = min_value('smoking')
max_value_smoking = max_value('smoking')

El valor min smoking es 0
El valor max smoking es 1


# Split train - test data

In [20]:
df_train = df_train(275)
#print(df_train)
df_train

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,target
0,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
1,55,0,7861,0,38,0,263358,1.1,136,1,0,6,1
2,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
3,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
4,65,1,160,1,20,0,327000,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,44,0,582,1,30,1,263358,1.6,130,1,1,244,0
271,51,0,582,1,40,0,221000,0.9,134,0,0,244,0
272,67,0,213,0,38,0,215000,1.2,133,0,0,245,0
273,42,0,64,0,40,0,189000,0.7,140,1,0,245,0


In [60]:
#df_test = df_test(25)
#df_test

# Creating Generative Bayesian model

In [22]:
# Split predictors variables(x) and target variable(y)

x_real = df_train.drop('target', axis=1)
y_real = df_train['target']

In [23]:
# verify matrix 
x_shape = x_real.shape
y_shape = y_real.shape

#x_shape

## Split data

In [45]:
#binary data

binary_data = x_real[['anaemia','diabetes','high_blood_pressure','sex','smoking']]

# array only values
binary_data_values = binary_data.values

(275, 5)

In [47]:
#numeric data

numeric_data = x_real[['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time']]

# array only values
numeric_data_values = numeric_data.values

In [48]:
# target variable
y_target = y_real
print(y_target)

0      1
1      1
2      1
3      1
4      1
      ..
270    0
271    0
272    0
273    0
274    0
Name: target, Length: 275, dtype: int64


#### Normal model bayesian
with pm.Model() as generative_model:
    x = pm.Normal('x', mu=x_real.mean(), sd=1.0, shape=(275, 12))
    x_sintetico = pm.Normal('x_sintetico', mu=x, sd=1.0, shape=(275,12))
    
    y = pm.Normal('y', mu=y_real.mean(), sd=1.0, shape=(275))
    y_sintetico = pm.Normal('y_sintetico', mu=y, sd=1.0, shape=(275))

In [54]:
with pm.Model() as generative_model:
    x_binaria = pm.Bernoulli('x_binaria',p=binary_data, shape=binary_data_values.shape, observed=binary_data)
    x_numeric = pm.HalfNormal('x_numeric',sd=1, shape=numeric_data_values.shape)
    y_target = pm.Bernoulli('y_target',p=y_target, shape=y_target.shape, observed=y_target)

In [112]:
# generating 
with generative_model:
    samplers = pm.sample(1000, tune=1000)

/home/stefy/anaconda3/envs/ml-env/lib/python3.7/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [y_sintetico, y, x_sintetico, x]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 40 seconds.


In [113]:
x_sintetico_generado = samplers['x_sintetico']
y_sintetico_generado = samplers['y_sintetico']

In [114]:
x_sintetico_generado

array([[[ 6.12602803e+01,  1.52104954e+00,  5.68455943e+02, ...,
          8.37519025e-01,  1.15334354e+00,  1.18721155e+02],
        [ 6.01313737e+01,  2.31218801e+00,  5.64890884e+02, ...,
          1.80868009e+00,  3.84564288e-01,  1.17056574e+02],
        [ 6.16733511e+01,  5.09753933e-01,  5.66374374e+02, ...,
          2.37728669e+00, -1.37395038e+00,  1.18596658e+02],
        ...,
        [ 6.09836542e+01,  1.52983862e+00,  5.68253602e+02, ...,
         -1.09370409e+00,  5.91037540e-01,  1.19118303e+02],
        [ 5.94875046e+01,  1.10278213e+00,  5.67258222e+02, ...,
         -1.63863796e-01,  1.08804274e+00,  1.18111559e+02],
        [ 6.38170538e+01, -1.34835004e-01,  5.65805219e+02, ...,
          2.49122797e+00,  1.63700687e+00,  1.19672159e+02]],

       [[ 6.17378705e+01,  1.48791096e+00,  5.69083931e+02, ...,
          2.09311608e+00,  2.27163979e+00,  1.19254808e+02],
        [ 5.97153748e+01,  1.91802230e+00,  5.65680983e+02, ...,
          1.83414434e+00,  7.20976787e

In [40]:
x_sintetico_generado = x_sintetico_generado.reshape(-1, x_sintetico_generado.shape[-1])
sintetico = pd.DataFrame(x_sintetico_generado, columns=['age','anaemia','creatinine_phosphokinase','diabetes','ejection_fraction','high_blood_pressure','platelets','serum_creatinine','serum_sodium','sex','smoking','time'])
sintetico

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,60.208068,0.142342,566.916569,-0.368208,41.186522,0.183256,261119.330661,2.616543,136.993542,-1.217449,0.227919,120.018485
1,62.566816,-1.326065,568.620833,-0.387803,38.943685,0.440658,261122.840841,1.904681,135.550406,-0.086672,3.996358,119.243160
2,61.846654,0.108774,568.379629,-0.392232,37.287571,-1.989640,261122.710992,0.071295,135.791962,0.236606,-1.554355,119.236261
3,63.346247,-0.773751,567.339842,-0.218774,39.121180,-1.535347,261120.923899,1.254420,135.740399,-1.242729,0.453695,118.679505
4,62.103876,1.142629,568.844970,-0.517234,38.750496,1.294688,261121.338449,0.851607,136.392517,1.052513,-1.056577,119.278451
...,...,...,...,...,...,...,...,...,...,...,...,...
1099995,59.685802,3.096356,568.984631,-0.258897,36.384373,-1.275860,261122.824247,3.937308,137.177204,0.983517,1.469569,116.574717
1099996,59.324731,-0.587094,566.134011,0.214621,39.610048,-1.284857,261121.344425,2.260907,137.216554,2.730124,1.183572,120.293992
1099997,62.110963,1.133512,569.027333,-2.148547,39.404680,2.120052,261122.005993,1.147622,136.416707,-0.021786,2.881537,117.710310
1099998,59.724944,1.492129,567.832387,1.605508,38.161033,-0.366150,261122.109752,1.007190,135.812215,1.605936,-0.433749,118.232693


# Analyze generic data 

## predictor variables

### Numeric data

In [115]:
data_gen_min_value_age = data_gen_min_value('age')
data_gen_max_value_age = data_gen_max_value('age')

El valor min age es 53.0537096662569
El valor max age es 67.64806379774963


In [116]:
data_gen_min_value_creatine = data_gen_min_value('creatinine_phosphokinase')
data_gen_max_value_creatine = data_gen_max_value('creatinine_phosphokinase')

El valor min creatinine_phosphokinase es 560.3266277006968
El valor max creatinine_phosphokinase es 574.2960499722319


In [117]:
data_gen_min_value_ejection = data_gen_min_value('ejection_fraction')
data_gen_max_value_ejection = data_gen_max_value('ejection_fraction')

El valor min ejection_fraction es 31.148686550497267
El valor max ejection_fraction es 45.45155967290858


In [118]:
data_gen_min_value_platelets = data_gen_min_value('platelets')
data_gen_max_value_platelets = data_gen_max_value('platelets')

El valor min platelets es 261113.33859230817
El valor max platelets es 261128.060138904


In [119]:
data_gen_min_value_creatine = data_gen_min_value('serum_creatinine')
data_gen_max_value_creatine = data_gen_max_value('serum_creatinine')

El valor min serum_creatinine es -5.395561495471548
El valor max serum_creatinine es 8.619566090281234


In [120]:
data_gen_min_value_sodium = data_gen_min_value('serum_sodium')
data_gen_max_value_sodium = data_gen_max_value('serum_sodium')

El valor min serum_sodium es 129.32813028548557
El valor max serum_sodium es 143.66755550830155


In [121]:
data_gen_min_value_time = data_gen_min_value('time')
data_gen_max_value_time = data_gen_max_value('time')

El valor min time es 112.35982669483076
El valor max time es 126.19628930290399


### Binary data

In [122]:
data_gen_min_value_anaemia = data_gen_min_value('anaemia')
data_gen_max_value_anaemia = data_gen_max_value('anaemia')

El valor min anaemia es -6.298980452805405
El valor max anaemia es 7.319510336736357


In [123]:
data_gen_min_value_diabetes = data_gen_min_value('diabetes')
data_gen_max_value_diabetes = data_gen_max_value('diabetes')

El valor min diabetes es -6.27203182928784
El valor max diabetes es 7.137233504617402


In [124]:
data_gen_min_value_pressure = data_gen_min_value('high_blood_pressure')
data_gen_max_value_pressure = data_gen_max_value('high_blood_pressure')

El valor min high_blood_pressure es -6.409483925563197
El valor max high_blood_pressure es 7.087408197845221


In [125]:
data_gen_min_value_sex = data_gen_min_value('sex')
data_gen_max_value_sex = data_gen_max_value('sex')

El valor min sex es -6.082764628465771
El valor max sex es 8.087629638977758


In [126]:
data_gen_min_value_smoking = data_gen_min_value('smoking')
data_gen_max_value_smoking = data_gen_max_value('smoking')

El valor min smoking es -6.168863243509936
El valor max smoking es 7.0874533091012655


## Target variable

In [27]:
y_sintetico_generado

array([[-1.43431473,  0.57013016, -1.88297449, ...,  3.7113344 ,
         0.80205195, -1.53162098],
       [ 0.1463058 ,  0.86717583,  2.06923482, ..., -2.39621955,
         0.55928071,  0.51095486],
       [-1.92643755, -1.55973937,  1.61326229, ..., -0.07923502,
         0.64705781,  2.19021291],
       ...,
       [ 3.26041646,  3.16264139,  0.96100516, ...,  0.85012344,
        -0.88635232, -0.89789306],
       [-2.71983532, -2.17781292, -0.16297358, ..., -1.05363783,
         0.83221564,  0.8024035 ],
       [ 2.69893361,  2.29557481, -0.29156242, ...,  1.98380905,
         0.37816809, -0.12404794]])

In [28]:
#y_sintetico_generado = y_sintetico_generado.reshape(-1, y_sintetico_generado.shape[-1])
y_sintetico_generado = y_sintetico_generado.flatten()
sintetic = pd.DataFrame({'target': y_sintetico_generado})
sintetic

,target
0,-1.434315
1,0.570130
2,-1.882974
3,1.673105
4,-2.973259
...,...
1099995,2.092998
1099996,1.584668
1099997,1.983809
1099998,0.378168


# Otro modelo generativo

In [ ]:
# generating 
with generative_model:
    samplers = pm.sample(1000, tune=1000)